Lingyi Xu (lingyix2) CS398 HW3

# Import Libraries

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from random import randint

# Fetch Data

In [45]:
# code from lecture notes
MNIST_data = h5py.File('MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:] )
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] )
y_test = np.int32( np.array( MNIST_data['y_test'][:,0] ))

# Parameters of Data

In [46]:
num_outputs = 10
d = 28
kx = 3
ky = 3

# CNN Network

In [50]:
class cnn():
    #constructor-----------------------------------------------------------------
    def __init__(self,X,y,input_size,output_size,d,k=3,num_channel = 1,learning_rate=0.01,batch_size=150):
        self.X = X
        self.y = y
        self.k = k
        self.d = d
        self.W = np.random.randn(output_size,(d - k+1),(d-k+1),num_channel)*0.01
        self.b = np.zeros((output_size,1))
        self.kernel = np.random.randn(k,k,num_channel) * 0.01
        self.H = np.zeros((d - k+1,d-k+1,num_channel))
        self.Z = np.zeros((d - k+1,d-k+1,num_channel))
        self.output_size = output_size
        self.num_channel = num_channel
        self.learning_rate = learning_rate
        self.batch_size = batch_size
    
    #helper functions in building cnn-network-----------------------------------
    def softmax(self, z):
        return np.exp(z)/np.sum(np.exp(z))

    def relu(self, x):
        return np.maximum(0,x)

    def conv(self,img,k,d,ky,kx):
        x = d-kx+1
        y = d-ky+1
        result = np.zeros((y,x))
        for i in range(y):
            for j in range(x):
                result[i][j] = np.sum(k*img[i:i+ky,j:j+ky])        
        return result
    
    def relu_back(self, z):
        return (z>0)
    
    #forward and backward propagation-------------------------------------------
    def forward_prop(self,x):
        k = self.k
        x = x.reshape((28,28))
        Z = np.zeros((d - k+1,d-k+1,self.num_channel))
        H = np.zeros((d - k+1,d-k+1,self.num_channel))
        for p in range(self.num_channel):
            Z[:,:,p] = self.conv(x,self.kernel[:,:,p],d, self.kernel[:,:,p].shape[0],self.kernel[:,:,p].shape[1])
            H[:,:,p] = self.relu(Z[:,:,p])
        self.H = H
        self.Z = Z
        temp = np.dot(self.W.reshape(self.output_size,np.prod(H.shape)),H.reshape(np.prod(H.shape)))
        U = temp.reshape((self.output_size,1)) + self.b
        p = self.softmax(U)
        return p
    
    def backward_prop(self,x,y,p):
        x = x.reshape((28,28))
        dU = -1.0*p
        dU[y] = dU[y] + 1.0
        db = dU
        delta = np.zeros((d - self.k+1,d-self.k+1,self.num_channel))
        dtemp = np.dot(self.W.reshape(self.output_size,np.prod(self.H.shape)).T,dU)
        dtemp = dtemp.reshape((d - self.k+1,d-self.k+1,self.num_channel))
        delta = dtemp
        H = self.H
        Z = self.Z
        dk = np.zeros((self.k,self.k,self.num_channel))
        for p in range(self.num_channel):
            dktemp = np.multiply(self.relu_back(Z[:,:,p]), delta[:,:,p] )
            dk[:,:,p] = self.conv(x, dktemp, d, dktemp.shape[0], dktemp.shape[1])
        temp_W = np.dot(dU.reshape(self.output_size,1), H.reshape(np.prod(H.shape),1).T).reshape(self.W.shape)
        return temp_W, db, dk
    
    #train function-------------------------------------------------------
    def train(self):
        num_epochs = 1000
        for epochs in range(num_epochs):
            LR = self.learning_rate/(0.01*epochs+1)
            total_correct = 0
            for n in range(self.batch_size):
                n_random = randint(0,len(x_train)-1 )
                y = y_train[n_random]
                x = x_train[n_random][:]
                p = self.forward_prop(x)
                prediction = np.argmax(p)
                if (prediction == y):
                    total_correct += 1
                temp_W, db, dk = self.backward_prop(x,y,p)
                self.W = self.W + LR*temp_W
                self.b = self.b + LR*db
                self.kernel = self.kernel + LR*dk
            if epochs%10 == 0:
                print("At iteration "+str(epochs)+", accuracy is: "+str(total_correct/self.batch_size))

# Train and Test

In [ ]:
LR = .01
num_epochs = 1000
network = cnn(x_train,y_train,784,10,28,3,5)
network.train()

In [49]:
correct = 0
for n in range(len(x_test)):
    y = y_test[n]
    x = x_test[n][:]
    prediction = np.argmax(network.forward_prop(x))
    if (prediction == y):
        correct += 1
print("The accuracy on Test data is:"+str(correct/np.float(len(x_test))))

The accuracy on Test data is:0.9705
